## Use Beautifulsoup to scrape the 2018 movies grosses table
### By Pam Zhao

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from login import user
from login import pwd

In [2]:
# url of the page
url = 'https://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr=2018&p=.htm'

# Retrieve page from the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')


In [3]:
# Examine the result of html
print(soup.prettify())


<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
 <head>
  <title>
   2018 Yearly Box Office Results - Box Office Mojo
  </title>
  <meta content="2018, year, yearly, box, office, result, list, movie, movies, listing, listings, top movies, all time, film" name="keywords"/>
  <meta content="Yearly box office results for 2018." name="description"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/>
 </head>
 <body>
  <iframe frameborder="0" height="1" id="sis_pixel_sitewide" marginheight="0" marginwidth="0" style="display: none;" width="1">
  </iframe>
  <script>
   setTimeout(function(){
        try{
            //sis3.0 pixel
            var cacheBust = Math.random() * 10000000000000000,
                url_sis3 = '//s.amazon-adsystem.com/iu3?

In [4]:
table = soup.find_all('table')
# Try and see which table has the grosses data
print(table[3])


<table border="0" cellpadding="5" cellspacing="1"><tr bgcolor="#dcdcdc"><td align="center"><font size="2"><a href="/yearly/chart/?view2=worldwide&amp;yr=2018&amp;sort=rank&amp;order=ASC&amp;p=.htm">Rank</a></font></td><td align="center"><font size="2"><a href="/yearly/chart/?view2=worldwide&amp;yr=2018&amp;sort=title&amp;order=ASC&amp;p=.htm">Title (click to view)</a></font></td><td align="center"><font size="2"><a href="/yearly/chart/?view2=worldwide&amp;yr=2018&amp;sort=studio&amp;order=ASC&amp;p=.htm">Studio*</a><br/><script language="javascript">
function MojoDrop1() {
var number = document.MojoDropDown1.mojolinks1.selectedIndex;location.href = document.MojoDropDown1.mojolinks1.options[number].value;}</script>
<table border="0" cellpadding="0" cellspacing="0">
<form name="MojoDropDown1"><tr><td>
<select name="mojolinks1" onchange="MojoDrop1(this.form)" style="font-family: Verdana; font-size: 8pt">
<option selected="">Filter</option><option value="/yearly/chart/?view2=worldwide&amp;

In [5]:
# Examine the tr row data
tblData = table[3]
tr = tblData.find_all('tr')
i = 0
for r in tr:
    print(f'TR#{i}**********{r}')
    i = i + 1
    

TR#0**********<tr bgcolor="#dcdcdc"><td align="center"><font size="2"><a href="/yearly/chart/?view2=worldwide&amp;yr=2018&amp;sort=rank&amp;order=ASC&amp;p=.htm">Rank</a></font></td><td align="center"><font size="2"><a href="/yearly/chart/?view2=worldwide&amp;yr=2018&amp;sort=title&amp;order=ASC&amp;p=.htm">Title (click to view)</a></font></td><td align="center"><font size="2"><a href="/yearly/chart/?view2=worldwide&amp;yr=2018&amp;sort=studio&amp;order=ASC&amp;p=.htm">Studio*</a><br/><script language="javascript">
function MojoDrop1() {
var number = document.MojoDropDown1.mojolinks1.selectedIndex;location.href = document.MojoDropDown1.mojolinks1.options[number].value;}</script>
<table border="0" cellpadding="0" cellspacing="0">
<form name="MojoDropDown1"><tr><td>
<select name="mojolinks1" onchange="MojoDrop1(this.form)" style="font-family: Verdana; font-size: 8pt">
<option selected="">Filter</option><option value="/yearly/chart/?view2=worldwide&amp;yr=2018&amp;sort=wwgross&amp;order=D

In [6]:
# Examine the td data from row 2 to 309
for i in range(2, 310):
    td = tr[i].find_all('td')
    row = [d.text for d in td]
    print(row)

['1', 'Avengers: Infinity War', 'BV', '$2,048.4', '$678.8', '33.1%', '$1,369.5', '66.9%']
['2', 'Black Panther', 'BV', '$1,346.9', '$700.1', '52.0%', '$646.9', '48.0%']
['3', 'Jurassic World: Fallen Kingdom', 'Uni.', '$1,309.5', '$417.7', '31.9%', '$891.8', '68.1%']
['4', 'Incredibles 2', 'BV', '$1,242.8', '$608.6', '49.0%', '$634.2', '51.0%']
['5', 'Aquaman', 'WB', '$1,147.8', '$335.1', '29.2%', '$812.7', '70.8%']
['6', 'Bohemian Rhapsody', 'Fox', '$903.2', '$216.4', '24.0%', '$686.7', '76.0%']
['7', 'Venom (2018)', 'Sony', '$855.0', '$213.5', '25.0%', '$641.5', '75.0%']
['8', 'Mission: Impossible - Fallout', 'Par.', '$791.1', '$220.2', '27.8%', '$571.0', '72.2%']
['9', 'Deadpool 2', 'Fox', '$778.9', '$318.5', '40.9%', '$460.5', '59.1%']
['10', 'Fantastic Beasts: The Crimes of Grindelwald', 'WB', '$653.7', '$159.6', '24.4%', '$494.1', '75.6%']
['11', 'Ant-Man and the Wasp', 'BV', '$622.7', '$216.6', '34.8%', '$406.0', '65.2%']
['12', 'Ready Player One', 'WB', '$582.9', '$137.7', '23.6

In [7]:
# Create an empty dataframe
df = []

# iterate through the table row data and save into a dataframe
for i in range(2, 310):
    td = tr[i].find_all('td')
    row = [d.text for d in td]
    df.append(row)   
    
grosses_df = pd.DataFrame(df, columns=['rank', 'title', 'studio', 'worldwide', 'domestic', 'dom_pct', 'overseas', 'os_pct'])

grosses_df

,rank,title,studio,worldwide,domestic,dom_pct,overseas,os_pct
0,1,Avengers: Infinity War,BV,"$2,048.4",$678.8,33.1%,"$1,369.5",66.9%
1,2,Black Panther,BV,"$1,346.9",$700.1,52.0%,$646.9,48.0%
2,3,Jurassic World: Fallen Kingdom,Uni.,"$1,309.5",$417.7,31.9%,$891.8,68.1%
3,4,Incredibles 2,BV,"$1,242.8",$608.6,49.0%,$634.2,51.0%
4,5,Aquaman,WB,"$1,147.8",$335.1,29.2%,$812.7,70.8%
5,6,Bohemian Rhapsody,Fox,$903.2,$216.4,24.0%,$686.7,76.0%
6,7,Venom (2018),Sony,$855.0,$213.5,25.0%,$641.5,75.0%
7,8,Mission: Impossible - Fallout,Par.,$791.1,$220.2,27.8%,$571.0,72.2%
8,9,Deadpool 2,Fox,$778.9,$318.5,40.9%,$460.5,59.1%
9,10,Fantastic Beasts: The Crimes of Grindelwald,WB,$653.7,$159.6,24.4%,$494.1,75.6%


In [8]:
grosses_df.dtypes

rank         object
title        object
studio       object
worldwide    object
domestic     object
dom_pct      object
overseas     object
os_pct       object
dtype: object

In [9]:
# Convert the rank data type to integer
grosses_df['rank'] = grosses_df['rank'].astype('int')

In [10]:
# Define a function to parse the money
def parse_money(row):
    money = row.replace("$", "")
    money = money.replace(",", "")
    if money[-1] == 'k':
        money = float(money[:-1])*1000
    else:
        money = float(money)*1000000
    return money

In [11]:
#grosses_df['ww_gross'] = grosses_df['worldwide'].apply(parse_money, axis=1)
grosses_df['ww_gross'] = grosses_df['worldwide'].apply(parse_money)
grosses_df['dom_gross'] = grosses_df['domestic'].apply(parse_money)
grosses_df['overseas'].replace("n/a", "0", inplace=True)
grosses_df['os_gross'] = grosses_df['overseas'].apply(parse_money)
grosses_df['dom_pct'] = grosses_df.dom_pct.str[:-1]
grosses_df['os_pct'] = grosses_df.os_pct.str[:-1]

In [12]:
grosses_df

,rank,title,studio,worldwide,domestic,dom_pct,overseas,os_pct,ww_gross,dom_gross,os_gross
0,1,Avengers: Infinity War,BV,"$2,048.4",$678.8,33.1,"$1,369.5",66.9,2.048400e+09,678800000.0,1.369500e+09
1,2,Black Panther,BV,"$1,346.9",$700.1,52.0,$646.9,48.0,1.346900e+09,700100000.0,6.469000e+08
2,3,Jurassic World: Fallen Kingdom,Uni.,"$1,309.5",$417.7,31.9,$891.8,68.1,1.309500e+09,417700000.0,8.918000e+08
3,4,Incredibles 2,BV,"$1,242.8",$608.6,49.0,$634.2,51.0,1.242800e+09,608600000.0,6.342000e+08
4,5,Aquaman,WB,"$1,147.8",$335.1,29.2,$812.7,70.8,1.147800e+09,335100000.0,8.127000e+08
5,6,Bohemian Rhapsody,Fox,$903.2,$216.4,24.0,$686.7,76.0,9.032000e+08,216400000.0,6.867000e+08
6,7,Venom (2018),Sony,$855.0,$213.5,25.0,$641.5,75.0,8.550000e+08,213500000.0,6.415000e+08
7,8,Mission: Impossible - Fallout,Par.,$791.1,$220.2,27.8,$571.0,72.2,7.911000e+08,220200000.0,5.710000e+08
8,9,Deadpool 2,Fox,$778.9,$318.5,40.9,$460.5,59.1,7.789000e+08,318500000.0,4.605000e+08
9,10,Fantastic Beasts: The Crimes of Grindelwald,WB,$653.7,$159.6,24.4,$494.1,75.6,6.537000e+08,159600000.0,4.941000e+08


In [13]:
grosses_df.dtypes

rank           int32
title         object
studio        object
worldwide     object
domestic      object
dom_pct       object
overseas      object
os_pct        object
ww_gross     float64
dom_gross    float64
os_gross     float64
dtype: object

In [14]:
grosses_df.head()

,rank,title,studio,worldwide,domestic,dom_pct,overseas,os_pct,ww_gross,dom_gross,os_gross
0,1,Avengers: Infinity War,BV,"$2,048.4",$678.8,33.1,"$1,369.5",66.9,2.048400e+09,678800000.0,1.369500e+09
1,2,Black Panther,BV,"$1,346.9",$700.1,52.0,$646.9,48.0,1.346900e+09,700100000.0,6.469000e+08
2,3,Jurassic World: Fallen Kingdom,Uni.,"$1,309.5",$417.7,31.9,$891.8,68.1,1.309500e+09,417700000.0,8.918000e+08
3,4,Incredibles 2,BV,"$1,242.8",$608.6,49.0,$634.2,51.0,1.242800e+09,608600000.0,6.342000e+08
4,5,Aquaman,WB,"$1,147.8",$335.1,29.2,$812.7,70.8,1.147800e+09,335100000.0,8.127000e+08


In [15]:
df = grosses_df.loc[:, ['rank', 'title', 'studio', 'ww_gross', 'dom_gross', 'dom_pct', 'os_gross', 'os_pct']]

In [16]:
df.head()

,rank,title,studio,ww_gross,dom_gross,dom_pct,os_gross,os_pct
0,1,Avengers: Infinity War,BV,2.048400e+09,678800000.0,33.1,1.369500e+09,66.9
1,2,Black Panther,BV,1.346900e+09,700100000.0,52.0,6.469000e+08,48.0
2,3,Jurassic World: Fallen Kingdom,Uni.,1.309500e+09,417700000.0,31.9,8.918000e+08,68.1
3,4,Incredibles 2,BV,1.242800e+09,608600000.0,49.0,6.342000e+08,51.0
4,5,Aquaman,WB,1.147800e+09,335100000.0,29.2,8.127000e+08,70.8


In [17]:
df.rename(columns={'ww_gross':'worldwide', 'dom_gross':'domestic', 'os_gross':'overseas'}, inplace=True)

In [18]:
df.head()

,rank,title,studio,worldwide,domestic,dom_pct,overseas,os_pct
0,1,Avengers: Infinity War,BV,2.048400e+09,678800000.0,33.1,1.369500e+09,66.9
1,2,Black Panther,BV,1.346900e+09,700100000.0,52.0,6.469000e+08,48.0
2,3,Jurassic World: Fallen Kingdom,Uni.,1.309500e+09,417700000.0,31.9,8.918000e+08,68.1
3,4,Incredibles 2,BV,1.242800e+09,608600000.0,49.0,6.342000e+08,51.0
4,5,Aquaman,WB,1.147800e+09,335100000.0,29.2,8.127000e+08,70.8


In [19]:
df.to_csv('grosses.csv', index=False)

In [20]:
# MySQL Connection
connection_string = f'{user}:{pwd}@127.0.0.1/movies_db'
engine = create_engine(f'mysql://{connection_string}')

In [24]:
engine.table_names()

['grosses', 'movies']

In [23]:
# Load the grosses_df to MySQL database
df.to_sql(name='grosses', con=engine, if_exists='append', index=False)

OperationalError: (MySQLdb._exceptions.OperationalError) (1054, "Unknown column 'worldwide' in 'field list'") [SQL: 'INSERT INTO grosses (`rank`, title, studio, worldwide, domestic, dom_pct, overseas, os_pct) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'] [parameters: ((1, 'Avengers: Infinity War', 'BV', 2048400000.0, 678800000.0, '33.1', 1369500000.0, '66.9'), (2, 'Black Panther', 'BV', 1346900000.0, 700100000.0, '52.0', 646900000.0, '48.0'), (3, 'Jurassic World: Fallen Kingdom', 'Uni.', 1309500000.0, 417700000.0, '31.9', 891800000.0, '68.1'), (4, 'Incredibles 2', 'BV', 1242800000.0, 608600000.0, '49.0', 634200000.0, '51.0'), (5, 'Aquaman', 'WB', 1147800000.0, 335100000.0, '29.2', 812700000.0, '70.8'), (6, 'Bohemian Rhapsody', 'Fox', 903200000.0, 216400000.0, '24.0', 686700000.0, '76.0'), (7, 'Venom (2018)', 'Sony', 855000000.0, 213500000.0, '25.0', 641500000.0, '75.0'), (8, 'Mission: Impossible - Fallout', 'Par.', 791100000.0, 220200000.0, '27.8', 571000000.0, '72.2')  ... displaying 10 o

In [ ]:
pd.read_sql_query('select * from grosses', con=engine).head()

## Conver Chris c_movies_omdb.csv file into Panadas DataFrame and then load into MySQL database

In [26]:
movies_df = pd.read_csv("c_movies_omdb.csv")

movies_df.head()

,title,rank,rating,imdbscore,movie_directors
0,Avengers: Infinity War,1,PG-13,8.5,"Anthony Russo, Joe Russo"
1,Black Panther,2,PG-13,7.3,Ryan Coogler
2,Jurassic World: Fallen Kingdom,3,PG-13,6.2,J.A. Bayona
3,Incredibles 2,4,PG,7.7,Brad Bird
4,Aquaman,5,PG-13,7.2,James Wan


In [27]:
movies_df.rename(columns={'movie_directors':'directors'}, inplace=True)

In [28]:
engine.table_names()

['grosses', 'movies']

In [29]:
movies_df.to_sql(name='movies', con=engine, if_exists='append', index=False)

OperationalError: (MySQLdb._exceptions.OperationalError) (1054, "Unknown column 'directors' in 'field list'") [SQL: 'INSERT INTO movies (title, `rank`, rating, imdbscore, directors) VALUES (%s, %s, %s, %s, %s)'] [parameters: (('Avengers: Infinity War', 1, 'PG-13', 8.5, 'Anthony Russo, Joe Russo'), ('Black Panther', 2, 'PG-13', 7.3, 'Ryan Coogler'), ('Jurassic World: Fallen Kingdom', 3, 'PG-13', 6.2, 'J.A. Bayona'), ('Incredibles 2', 4, 'PG', 7.7, 'Brad Bird'), ('Aquaman', 5, 'PG-13', 7.2, 'James Wan'), ('Bohemian Rhapsody', 6, 'PG-13', 8.1, 'Bryan Singer'), ('Venom', 7, 'PG-13', 6.8, 'Ruben Fleischer'), ('Mission: Impossible - Fallout', 8, 'PG-13', 7.8, 'Christopher McQuarrie')  ... displaying 10 of 304 total bound parameter sets ...  ('The Swan', 307, 'APPROVED', 6.5, 'Charles Vidor'), ('An Actor Prepares', 308, 'TV-14', 5.0, 'Steve Clark'))] (Background on this error at: http://sqlalche.me/e/e3q8)

In [25]:
pd.read_sql_query('select * from movies', con=engine).head()

,id,title,rank,rating,imdbscore,director
